<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>

# Test comparing deposition and erosion to dell size sensitvity 
This notebook was created by Sam Anderson, Mikey Sison and Angel Monsalve.

<hr>
<small>For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

**What is this notebook?**

This notebook generates results to compare the behaivor of ... to be completed later 

To test the ability of our component for predicting changes in the bed surface elevation we obtained an analytical solution for an idealized channel with uniform flow conditions. In this case, a given bed load transport rate is imposed at the upstream boundary such that the bed surface slope must adjust until the channel reaches a stable condition. We combined Manning’s equation to include uniform flow conditions and Meyer-Peter & Müller (1948) to estimate bed load transport rate within the channel. 

We conducted two tests to evaluate the response of our component. Both cases start with the same initial bed configuration but differ in the imposed upstream sediment supply rate. In general terms, they consist of a 1500 m long, straight channel with an initial bed surface slope of 0.015 m/m, the elevation at the outlet is fixed at 0 m, and the surface roughness is n = 0.03874. Flow discharge is constant Q = 100 m3/s and is specified by using a rainfall intensity of 0.01 m/s acting over a single cell of 100 m per side (∆x=∆y) located at the upstream boundary. The grid consists only of uniformly sized cells. The bed surface GSD is uniform with a grain size of 50 mm and , the bed load transport equation is that of Meyer-Peter & Müller (1948). In OverlandFlow we specified h_init, the initial water depth in all cells, as 1 mm, the time step is limited to a maximum of 5 s, and left all the other variables with are left as their default value. To better manage the interaction of boundary conditions between OverlandFlow and riverBedDynamics we added ghost nodes, one at the downstream and two and the upstream end. These  ghost nodes are used to extend the study reach and help reaching uniform flow conditions once a new stable slope has been reached. The modeled scenarios are a purely aggradation case in which q_b = 0.0087 m2/s and purely degradation case where q_b = 0.0012 m2/s. By applying Eq. 37, the equilibrium slope for both scenarios are 0.025 and 0.010, respectively.

We ran each case for 120 days and compared the predicted and analytical bed slopes in both cases at the end of the simulation. We chose 120 days as comparing point because the rate at which the bed elevations were changing were relatively small, 9·10-5 and -4·10-4 m/day in the aggradation and degradation cases, respectively. We considered these rates small enough to be representative of an equilibrium condition. 

In the aggradation case our LEM predicts an S equal to 0.0251 (percentage error of 0.32 %), the degradation case had an S of 0.0101 (percentage error of 1.439 %). Locally, the major differences between the predicted and analytical bed elevation are found in the upstream region, near where the sediment supply is imposed. The maximum local differences are 0.136 m for the aggradation case and 0.200 m for the degradation scenario (Figure 7). The small percentage error and the general trend of the local surface elevation   (Figure 7) suggests that our component can accurately predict changes in bed elevation. 

We analyzed the sensitivity of our results to the mesh size in the pure aggradation case by comparing the bed elevation after 120 days of simulation using meshes with half and a quarter of the size of the beforementioned case (Figure 8). By the end of the run the average slope was 0.0251 in all cases. The percentage error was 0.32, 0.31, and 0.29 %   for the 25, 50, and 100 m grid resolution, respectively. Beside the mesh size, the configuration was identical in all cases except for the maximum time step, which was 5 s for the 100 and 50 m cases and 2.5 s for the 25 m run. 

More background on the model used here and the results presented will be published in the future and the citation will be added here:

**TITLE HERE**

The code used in this exercise is taken from the above reference.



**Now on to the code.**
* Below we import Landlab components, functions for importing data, numpy and plotting tools. You should not need to change this.

In [1]:
## Code Block 1

%reset -f
import numpy as np
import pandas as pd
import copy
from matplotlib import pyplot as plt
from landlab.components import OverlandFlowSpatiallyVariableInputs, RiverBedDynamics
from landlab.io import read_esri_ascii
from landlab import imshow_grid


Now we import the elevation data (DEM), as well as grain size distribution (GSD) information. There are three possible DEM's which can be used, the default 'bedElevationDEM.asc' for the cases of either erosion or depostion, and either 'bedElevationDEM50mMesh.asc' or 'bedElevationDEM25mMesh.asc' where mesh size is half or a quater of the size of the default, respectively.

In [ ]:
## Code Block 2

bedElevation =      'bedElevationDEM.asc'     # The other DEMs are 'bedElevationDEM50mMesh.asc' or 'bedElevationDEM25mMesh.asc'
(rmg, z) = read_esri_ascii(bedElevation, name='topographic__elevation')

gsd = pd.read_excel('bedGSD.xlsx', sheet_name='GSD', skiprows = 0).values   # GSD is same between all examples

And now we imput some model parameters. Note Max_dt can change with different DEM.

In [ ]:
dtPrecision = 3         # Avoids rounding errors
max_dt = 5              # Change to 2.5 if using the 25m mesh 
tPlot = 950400          # Plots will be obtained every this seconds **THIS VALUE WAS CHANGED FROM THE ORIGINAL**
storeData = 86400       # Stores results every this time
tmax = 121*86400        # Maximum simulation time
n = 0.03874             # Manning's n 

Here, we specify some important locations (links and nodes) where we take data from and where we specify where sediment is coming from. Note: These values can change with different DEMs.

In [ ]:
# Flow, bed, and upstream simulation conditions 
upstreamSedSupply = -0.0012         # bedload rate at inlet

# Link Id in which sediment supply and discharge enters
link_Inlet = np.array([83])

# Node Id where Water depth is specified
Node_Inlet = np.array([52])

# Node ID for fixed Nodes
fixedNodesId = np.array((1,4))

In [ ]:
## Code Block 3

# Creates fields and instantiate the component
OverlandFlowSpatiallyVariableInputs.input_var_names
RiverBedDynamics.input_var_names
rmg.add_zeros('bed_surface__roughness', at = 'link')
rmg.add_zeros('surface_water__depth', at = 'node')
rmg.add_zeros('rainfall__intensity', at = 'node')
rmg['node']['bed_surface__grain_size_distribution_location'] = np.zeros_like(z)     
rmg['node']['topographic__elevation_original'] = copy.deepcopy(z)               # Converts precipitation from mm/hr to m/s
rmg['node']['bed_surface__grainSizeDistribution_location'] = np.zeros_like(z)     

In [ ]:
## Code Block 4

# Instantiation of components
of = OverlandFlowSpatiallyVariableInputs(rmg, dt_max=max_dt,h_init=0.001)
rbd = RiverBedDynamics(rmg , gsd = gsd, variableCriticalShearStress = True, outletBoundaryCondition='fixedValue')

In [ ]:
## Code Block 6

# Set boundaries as closed boundaries, the outlet is set to an open boundary. 
rmg.set_watershed_boundary_condition(z) 


In [ ]:
## Code Block 7

# Creates the fixed nodes information
fixedNodes = np.zeros_like(z)   # fixedNodes defines as 1 if a node is fixed or 0 if it can vary in elevation
fixedNodes[fixedNodesId] = 1
rmg['node']['bed_surface__fixedElevation'] = fixedNodes                         # Assigns fixed locations to landlab grid 

# Create bed and flow initial condition
rmg['link']['bed_surface__roughness'] = np.zeros(rmg.number_of_links) + n       # n is Manning's roughness coefficient
rmg['node']['rainfall__intensity'][Node_Inlet] = 0.01                           # Precipitation in m/s
rmg['link']['sediment_transport__imposed_sediment_supply'][link_Inlet] = upstreamSedSupply


In [ ]:
## Code Block 8

# Node ID for calculated node elevation
calcNodesId = np.arange(51,57)
nCols = rmg.number_of_node_columns
nRowsCalcNodes = int(calcNodesId.shape[0]/nCols)
calcNodesId = np.reshape(calcNodesId,(nRowsCalcNodes,nCols))

In [1]:
## Code Block 9

""" Defines some variables to store data """
t = 0                                   # Initializates the variable
storeNow = True
plotNow = True                          # Used to save the plot at time zero
check_tmax = True
tPlotOrg=copy.deepcopy(tPlot)           # A copy of tPlot
storeDataOrg=copy.deepcopy(storeData)   # A copy of tPlot

NameError: name 'copy' is not defined

In [ ]:
## Code Block 10

while t < tmax:
    
    of.overland_flow()  # Runs overland flow for one time step
    rbd.run_one_step()  # Runs riverBedDynamics for one time step
    
    # Gradient preserving at upstream ghost cells
    dsNodesId = np.array(calcNodesId[0,1]-np.arange(1,3)*nCols)
    z = rmg['node']['topographic__elevation']
    bedSlope = (z[dsNodesId[0]] - z[dsNodesId[1]]) / rmg.dx
    
    for i in np.arange(0,calcNodesId.shape[0]):
        rmg['node']['topographic__elevation'][calcNodesId[i,1:nCols-1]] = z[calcNodesId[i,1:nCols-1] - 2*nCols] + 2 * rmg.dx * bedSlope
        
    ## Stores results
    storeData = round(storeData-of.dt, dtPrecision)
    if (storeData <=0) or storeNow:
        print('Storing results at time :',np.round(t,1),' s')
        print('Upstream sediment supply :',upstreamSedSupply,' m2/s')
        data = np.reshape(np.hstack([t,(np.abs(of._q * rmg.dx).T)]),[1,rmg.number_of_links+1])
        with open("output0_links_surface_water__discharge.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')
        data = np.reshape(np.hstack([t,(of._h.T)]),[1,rmg.number_of_nodes+1])
        with open("output1_node_surface_water__depth.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')      
        data = np.reshape(np.hstack([t,np.abs(rbd._tau.T)]),[1,rmg.number_of_links+1])
        with open("output2_link_surface_water__shearStress.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')   
        data = np.reshape(np.hstack([t,rmg.at_node["topographic__elevation"].T]),[1,rmg.number_of_nodes+1])
        with open("output3_node_topographic__elevation.txt", "ab") as f:
            np.savetxt(f, data,'%.3f') 
        data = np.reshape(np.hstack([t,rmg.at_node["bed_surface__medianSize"].T]),[1,rmg.number_of_nodes+1])
        with open("output4_node_bed_surface__medianSize.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')
        data = np.reshape(np.hstack([t,rmg.at_link['sediment_transport__bedloadRate'].T]),[1,rmg.number_of_links+1])
        with open("output5_links_sediment_transport__bedloadRate.txt", "ab") as f:
            np.savetxt(f, data,'%.5f')  
        storeData = round(storeDataOrg, dtPrecision)
        storeNow = False

    tPlot = round(tPlot-of.dt, dtPrecision)
    if tPlot <= 0  or plotNow:
        print('Elapsed time :',np.round(t,1),' s. Current dt =',\
              np.round(of.dt,1),'. Adaptive time =',np.round(of._adaptive_dt,1),' s - Saving plot')
        
        # Water depth plot
        plot_name='Surface water depth [m] at ' + str(np.round(t,0)) + ' sec'
        imshow_grid(rmg, 'surface_water__depth',cmap='Blues',vmin=0,vmax=0.5,plot_name=plot_name)
        output='depth_'+str(np.round(t,0))+'.png'
        plt.savefig(output,dpi=300); plt.close()  
        
        #Bed surface variation plot
        plot_name='Bed surface elevation variation [m] at ' + str(np.round(t,0)) + ' sec'
        ZVar = rmg.at_node["topographic__elevation"] - rmg.at_node['topographic__elevation_original'] 
        imshow_grid(rmg, ZVar,cmap='RdGy',vmin=0,vmax=25,plot_name=plot_name)
        output='topographicVariation_'+str(np.round(t,0))+'.png'
        plt.savefig(output,dpi=300); plt.close()    

        plotNow = False
        tPlot = tPlotOrg

    # Updating t
    if (t + of.dt > tmax) and check_tmax:
        of.dt = tmax - t
        t = tmax
        storeDataNow = True  
        plotNow = True
        check_tmax = False
    else:
        t = round(t + of.dt, dtPrecision)

Figure 7: Changes in bed surface elevation for a case of a) pure aggradation and b) pure degradation. The analytical solution corresponds to the equilibrium slope given by Eq. 37. Only one point or elevation per cell was used for plotting. The small differences in bed elevation after 40 and 120 days indicate that the systems are achieving an equilibrium state.

In [ ]:
###
"""

THIS IS OLD CODE FROM TUTORIAL ONE, I LEFT IT TO CHANGED LATER TO MAKE SOME NEW PLOTS


## Code Block 11

# Results
## Water depth at the end of the simulation
data = np.loadtxt('output1_node_surface_water__depth.txt')
nodesToSample = np.arange(rmg.number_of_node_columns+1,2*rmg.number_of_node_columns)
x = np.arange(0,(rmg.number_of_node_columns-1)*rmg.dx,rmg.dx)

hSample = data[-1,nodesToSample]

hAnalytical = np.zeros_like(x)+h_init
hAnalytical[np.where(x<u*tmax)] =(-7/3 * (n**2 * u**2 * (x[np.where(x<u*tmax)]-u*tmax)))**(3/7)

plt.figure(1)
plt.plot(x, hSample, color='mediumblue')
plt.plot(x, hAnalytical, color='black')
plt.ylabel('Water depth (m)')
plt.xlabel('x (x)')
plt.ylim(0,2.5)
plt.xlim(0,5000)
plt.title('Water depth at 3600 s')
plt.savefig('WaterDepth.png',dpi=300);plt.close()
"""  